# LeetCode Medium

- 字符串中的‘ ’并不为 None，判断时要注意，if ‘ ’总会进入条件

In [8]:
s = ' '
for x in s:
    if x:
        print('yes')

yes


## NO.8 String to Integer (atoi)

题目：

* Implement atoi which converts a string to an integer.

* The function first discards as many whitespace characters as necessary until the first non-whitespace character is found. Then, starting from this character, takes an optional initial plus or minus sign followed by as many numerical digits as possible, and interprets them as a numerical value.

* The string can contain additional characters after those that form the integral number, which are ignored and have no effect on the behavior of this function.

* If the first sequence of non-whitespace characters in str is not a valid integral number, or if no such sequence exists because either str is empty or it contains only whitespace characters, no conversion is performed.

* If no valid conversion could be performed, a zero value is returned.

* Note:

    * Only the space character ' ' is considered as whitespace character.
    
    * Assume we are dealing with an environment which could only store integers within the 32-bit signed integer range: [−231,  231 − 1]. If the numerical value is out of the range of representable values, INT_MAX (231 − 1) or INT_MIN (−231) is returned.
    
题解：

* 依照题目，初步思路是双指针搜索，然而非常困难

* 更好的方式是正则表达式，又一个盲点（upset）

* '[\+\-]?\d+'

    - ^：匹配字符串开头
    
    - [\+\-]：代表一个+字符或-字符

    - ?：前面一个字符可有可无

    - \d：一个数字

    - +：前面一个字符的一个或多个

    - \D：一个非数字字符

    - \*：前面一个字符的0个或多个
    
* str.lstrip()

    - 脱去左侧空格并返回新字符串

* \*：解包，把列表的元素逐个返回

In [1]:
import re
class Solution:
    def myAtoi(self, s) -> int:
        return max(min(int(*re.findall('[\+\-]?\d+', s)), 2**31 - 1), -2**31)
a = Solution()
a.myAtoi("I'm born in -1996")

-1996

## NO.15 3Sum

题目：

* Given an array nums of n integers, are there elements a, b, c in nums such that a + b + c = 0? Find all unique triplets in the array which gives the sum of zero.

* Note:

    * The solution set must not contain duplicate triplets.
    
题解：

* nums 先排序，后哈希表优化，这种方法的时间空间复杂度都是$O(n^2)$

* 亮点是一次循环构建哈希表并且判断是否满足条件的处理，利用了成对的数的性质

* 然而这种方法依旧是时间超出无法完成测试

In [4]:
class Solution:
    def threeSum(self, nums):
        nums = sorted(nums)
        l = []
        for i,each in enumerate(nums[:-2]):
            if each > 0:
                break
            d = {}
            for k, x in enumerate(nums[i+1:]):
                if d.get(0-each-x) != None:
                    r = sorted([each,x,0-each-x])
                    if r not in l:
                        l.append(r)
                d[x] = k
        return l
a=Solution()
a.threeSum([-1, 0, 1, 2, -1, -4])

[[-1, 0, 1], [-1, -1, 2]]

* 更好的解法是排序加对撞指针 while循环

* 这种方式也特别巧妙，利用了已排序的列表的性质，指针位置的数字之和若大于0，尾部指针前移，否则首部指针后移

* sorted(reverse=True)    #逆序

In [1]:
class Solution:
    def threeSum(self, nums):
        nums.sort()
        n = len(nums)
        res = []
        #print(nums)
        for i in range(n):
            if i > 0 and nums[i] == nums[i-1]:   #加速循环，优化步骤
                continue
            left = i + 1
            right = n - 1
            while left < right:    #双指针常用while循环！
                cur_sum = nums[i] + nums[left] + nums[right]
                if cur_sum == 0:
                    tmp = [nums[i],nums[left],nums[right]]
                    res.append(tmp)
                    while left < right and nums[left] == nums[left+1]:    #优化步骤
                        left += 1
                    while left < right and nums[right] == nums[right-1]:    #优化步骤
                        right -= 1
                    left += 1
                    right -= 1
                elif cur_sum > 0:
                    right -= 1
                else:
                    left += 1
        return res
a=Solution()
a.threeSum([-1, 0, 1, 2, -1, -4])

[[-1, -1, 2], [-1, 0, 1]]

## NO.5 Longest Palindromic Substring

题目：

* Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.

题解：

* 最简单直观的就是暴力循环枚举判断,两次循环判断，加上回文字符的判断，时间复杂度为$O(n^3)$，由于只用了有限的变量储存数据，空间负责度是$O(1)$

In [3]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        length = 1
        for r in range(1,len(s)):
            for l in range(r):
                if s[l:r] == s[l:r][::-1] and r - l > length:
                    length = r - l
                    resault = s[l:r]
        return resault
a = Solution()
a.longestPalindrome("babad")

'bab'

* 一种改进的方法是动态规划，来优化判断回文数的过程，暴力算法中回文数的判断是翻转字符串，然而借助动态规划的思想可以不必每一次判断都翻转字符串，动态规划的状态是每一个子字符串是不是回文字符串，当新的字符串头尾相等时，则新的字符串就是回文字串。

* 用一个二维数组表示状态`[[True],[False]...]`

* 状态转移方程为 `dp[l, r] = (s[l] == s[r] and (r - l <= 2 or dp[l + 1, r - 1]))`

* 判断回文的过程复杂度得以降低，整体时间复杂度变为$O(n^2)$，由于需要储存状态，整体的空间复杂度也变为了$O(n^2)$

* 思路：用空间换时间

In [6]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        size = len(s)
        if size <= 1:
            return s
        # 二维 dp 问题
        # 状态：dp[l,r]: s[l:r] 包括 l，r ，表示的字符串是不是回文串
        # 设置为 None 是为了方便调试，看清楚代码执行流程
        dp = [[False for _ in range(size)] for _ in range(size)]

        longest_l = 1
        res = s[0]

        # 因为只有 1 个字符的情况在最开始做了判断
        # 左边界一定要比右边界小，因此右边界从 1 开始
        for r in range(1, size):
            for l in range(r):
                # 状态转移方程：如果头尾字符相等并且中间也是回文
                # 在头尾字符相等的前提下，如果收缩以后不构成区间（最多只有 1 个元素），直接返回 True 即可
                # 否则要继续看收缩以后的区间的回文性
                # 重点理解 or 的短路性质在这里的作用
                if s[l] == s[r] and (r - l <= 2 or dp[l + 1][r - 1]):
                    dp[l][r] = True
                    cur_len = r - l + 1
                    if cur_len > longest_l:
                        longest_l = cur_len
                        res = s[l:r + 1]
            # 调试语句
            for item in dp:
                print(item)
            print('---')
        return res
a = Solution()
a.longestPalindrome("babad")

[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
---
[False, False, True, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
---
[False, False, True, False, False]
[False, False, False, True, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
---
[False, False, True, False, False]
[False, False, False, True, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
---


'bab'

## NO.2 Add Two Numbers

题目：

* You are given two non-empty linked lists representing two non-negative integers. The digits are stored in reverse order and each of their nodes contain a single digit. Add the two numbers and return it as a linked list.

* You may assume the two numbers do not contain any leading zero, except the number 0 itself.

* Example:
    
    * Input: (2 -> 4 -> 3) + (5 -> 6 -> 4)

    * Output: 7 -> 0 -> 8

    * Explanation: 342 + 465 = 807.
    
题解：

* 对应位置的节点相加，然后依次进位

In [1]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, x):
#         self.val = x
#         self.next = None

class Solution:
    def addTwoNumbers(self, l1, l2):
        target = ListNode(0)   # 目标链表的根节点
        p = target
        add = 0    # 进位
        while l1 and l2:
            p.next = ListNode((l1.val+l2.val+add)%10)
            add = (l1.val+l2.val+add)//10
            p, l1, l2 = p.next, l1.next, l2.next
        #以上是两个链表的公共拥有的部分
        #下面在添加两个链表中较长的那个的私有部分
        l1 = l1 if l1 else l2
        while add:
            if l1:
                p.next = ListNode((l1.val+add)%10)
                add = (l1.val+add)//10
                p, l1 = p.next, l1.next
            else:
                p.next = ListNode(add)
                p = p.next
                break
        p.next = l1 
        
        return target.next

## NO.11 Container With Most Water

题目：

* Given n non-negative integers a1, a2, ..., an , where each represents a point at coordinate (i, ai). n vertical lines are drawn such that the two endpoints of line i is at (i, ai) and (i, 0). Find two lines, which together with x-axis forms a container, such that the container contains the most water.

* Note: You may not slant the container and n is at least 2.

* The above vertical lines are represented by array [1,8,6,2,5,4,8,3,7]. In this case, the max area of water (blue section) the container can contain is 49.

题解：

* 暴力双循环,时间复杂度为$O(n^2)$,空间复杂度$O(n)$

In [2]:
class Solution:
    def maxArea(self, height) -> int:
        ret = 0
        for r in range(1,len(height)):
            for l in range(r):
                tmp = (r-l)*min(height[l],height[r])
                ret = tmp if tmp > ret else ret
        return ret
a = Solution()
a.maxArea([1,8,6,2,5,4,8,3,7])

49

双指针法 指针移动规则与证明： 

- 每次选定围成水槽两板高度 h[i],h[j] 中的短板，向中间收窄 1格。

- 设每一状态下水槽面积为$ S(i, j),(0 <= i < j < n)$，由于水槽的实际高度由两板中的短板决定，则可得面积公式 $S(i, j) = min(h[i], h[j]) × (j - i)$。在每一个状态下，无论长板或短板收窄 1 格，都会导致水槽 底边宽度 -1：

    - 若向内移动短板，水槽的短板 $min(h[i], h[j])$ 可能变大，因此水槽面积 $S(i, j)$ 可能增大。

    - 若向内移动长板，水槽的短板 $min(h[i], h[j])$ 不变或变小，下个水槽的面积一定小于当前水槽面积。因此，向内收窄短板可以获取面积最大值。换个角度理解：
    
        - 若不指定移动规则，所有移动出现的 $S(i, j)$ 的状态数为 $C(n, 2)$，即暴力枚举出所有状态。
        - 在状态 $S(i, j)$ 下向内移动短板至 $S(i + 1, j)$（假设 h[i] < h[j] ），则相当于消去了 $S(i, j - 1), S(i, j - 2), ... , S(i, i + 1)$状态集合。而所有消去状态的面积一定 $<= S(i, j)$：
        - 短板高度：相比 $S(i, j)$ 相同或更短；
        - 底边宽度：相比 $S(i, j)$ 更短。
        - 因此所有消去的状态的面积都 $< S(i, j)$。通俗的讲，我们每次向内移动短板，所有的消去状态都不会导致丢失面积最大值 。

复杂度分析：

* 时间复杂度为$O(n)$，空间复杂度$O(1)$

In [1]:
class Solution:
    def maxArea(self, height) -> int:
        l,r = 0,len(height)-1
        ret = 0
        while l != r:
            tmp = (r - l) * min(height[l], height[r])
            ret = tmp if tmp > ret else ret
            if height[l]<height[r]:
                l += 1
            else:
                r -= 1
        return ret
a = Solution()
a.maxArea([1,8,6,2,5,4,8,3,7])

49

## NO.16 3Sum Closest

题目：

* Given an array nums of n integers and an integer target, find three integers in nums such that the sum is closest to target. Return the sum of the three integers. You may assume that each input would have exactly one solution.

题解：

* 这是承接15题的3数和问题，解法依然是排序加对撞指针，当序列有序之后就有了很多特殊的性质，比如依次循环，初始指针位置的数的加和一定会依次增大。

* 指针移动的规则也很有趣，从两端向中心移动，寻找目标，比目标大则右指针左移，其他情况则左指针右移，最后一定能够让指针重合退出循环。这也是由有序数列的特殊性质决定的。

* 用一个变量储存符合规定的数值，最后返回该变量即可。

In [1]:
class Solution:
    def threeSumClosest(self, nums, target: int) -> int:
        nums = sorted(nums)
        tmp = float("inf")
        for i, each in enumerate(nums[:-2]):
            l, r = i + 1, len(nums)-1
            while l != r:
                s = each + nums[l] + nums[r]
                if abs(s-target) <= tmp:
                    tmp = abs(s - target)
                    ret = s
                    a, b, c = each, nums[l], nums[r]
                if s < target:
                    l += 1
                else:
                    r -= 1
        return ret, [a, b, c]
a = Solution()
a.threeSumClosest([-1,2,1,-4],1)

(2, [-1, 1, 2])

## NO. 33 Search in Rotated Sorted Array

题目：

- Suppose an array sorted in ascending order is rotated at some pivot unknown to you beforehand.(i.e., [0,1,2,4,5,6,7] might become [4,5,6,7,0,1,2]).

- You are given a target value to search. If found in the array return its index, otherwise return -1.You may assume no duplicate exists in the array.

- Your algorithm's runtime complexity must be in the order of $O(log n)$.

题解：

- 首先不考虑题目要求的问题，可以在$O(n)$的复杂度下，一次循环返回结果

In [3]:
class Solution:
    def search(self, nums, target: int) -> int:
        for i, each in enumerate(nums):
            if each == target:
                return i
        return -1
a = Solution()
a.search([4,5,6,7,0,1,2],0)

4

- 这样的做法没有利用有序列表的性质，利用列表的性质还可以改进该做法

- 二分查找法，时间复杂度为$O(log_2n)$，空间复杂度最小可为$O(1)$

- [二分查找法编程模版](https://leetcode-cn.com/problems/search-insert-position/solution/te-bie-hao-yong-de-er-fen-cha-fa-fa-mo-ban-python-/) 非常细致，值得详细钻研

In [6]:
class Solution:
    def search(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: int
        """
        def find_rotate_index(left, right):
            if nums[left] < nums[right]:
                return 0
            
            while left <= right:
                pivot = (left + right) // 2
                if nums[pivot] > nums[pivot + 1]:
                    return pivot + 1
                else:
                    if nums[pivot] < nums[left]:
                        right = pivot - 1
                    else:
                        left = pivot + 1
                
        def search(left, right):
            """
            Binary search
            """
            while left <= right:
                pivot = (left + right) // 2
                if nums[pivot] == target:
                    return pivot
                else:
                    if target < nums[pivot]:
                        right = pivot - 1
                    else:
                        left = pivot + 1
            return -1
        
        n = len(nums)
        
        if n == 0:
            return -1
        if n == 1:
            return 0 if nums[0] == target else -1 
        
        rotate_index = find_rotate_index(0, n - 1)
        
        # if target is the smallest element
        if nums[rotate_index] == target:
            return rotate_index
        # if array is not rotated, search in the entire array
        if rotate_index == 0:
            return search(0, n - 1)
        if target < nums[0]:
            # search on the right side
            return search(rotate_index, n - 1)
        # search on the left side
        return search(0, rotate_index)
a = Solution()
a.search([4,5,6,7,0,1,2],0)

4

- Python 内置的列表查询和返回引用特别快，可以击败 84% 的提交者。

In [5]:
class Solution:
    def search(self, nums, target: int) -> int:
        if target in nums:
            return nums.index(target)
        else:
            return -1
a = Solution()
a.search([4,5,6,7,0,1,2],0)

4